## Procedures, Triggers and Views

This notebook focuses on enhancing the functionality and usability of the Superstore Sales database through the implementation of **stored procedures**, **triggers**, and **views** in PostgreSQL. These components are designed to automate common tasks, simplify complex queries, and provide a more efficient way to analyze sales data.

- **Stored Procedures**: Encapsulate business logic for calculating total sales and updating promotions.
- **Triggers**: Automate the assignment of promotions during data insertion, ensuring consistency.
- **Views**: Provide a reusable and comprehensive summary of sales data, simplifying analysis.

By leveraging these database features, the project aims to improve data management, streamline workflows, and enable deeper insights into sales performance.

---

## Running Queries in pgAdmin

The following SQL queries are designed to explore the `superstore_sales` database. Before running these queries, the user must ensure the database is properly set up by following these steps:

### Setup Instructions:
1. Run the `main.py` script to create and load the initial `sales_data` table into PostgreSQL.
2. Open **pgAdmin** and connect to the `superstore_sales` database using the details below.

### Query Execution:
Once the database setup is complete, you can use the provided SQL queries in **pgAdmin**. 

### Database Details:
- **Database Name**: `superstore_sales`
- **Host**: `localhost`
- **Port**: `5432`
- **Table**: `sales_data`

---

## Stored Procedures

### 1. Update `promo_id` for a Specific Product Category and Date Range
This procedure updates the `promo_id` in the `sales` table for rows that match a specific product category and fall within the promotion's date range.

In [ ]:
CREATE OR REPLACE PROCEDURE update_promo_id_for_category(
    IN promo_id INT, 
    IN category_name VARCHAR(50), 
    IN start_date DATE, 
    IN end_date DATE)
LANGUAGE plpgsql
AS $$
BEGIN
    -- Update promo_id for sales that match the product category and date range
    UPDATE sales s
    SET promo_id = update_promo_id_for_category.promo_id
    FROM products p
    WHERE s.product_id = p.product_id
      AND p.category = update_promo_id_for_category.category_name
      AND s.order_date BETWEEN update_promo_id_for_category.start_date AND update_promo_id_for_category.end_date;

    -- Output the number of rows updated
    RAISE NOTICE 'Updated promo_id to % for % sales in category "%" between % and %', 
        promo_id, ROW_COUNT, category_name, start_date, end_date;
END;
$$;

- Calling the procedure (example with `promo_id` 2):

In [ ]:
CALL update_promo_id_for_category(2, 'Office Supplies', '2023-08-01', '2023-08-31');
-- Output: NOTICE:  Updated promo_id to 2 for 150 sales in category "Office Supplies" between 2023-08-01 and 2023-08-31

### 2. Calculate Total Sales for a Promotion
This procedure calculates the total sales for a given promotion (`promo_id`).

In [ ]:
CREATE OR REPLACE PROCEDURE calculate_total_sales_for_promotion_with_name(
    IN promo_id INT, 
    OUT promo_name VARCHAR(100), 
    OUT total_sales DECIMAL(10, 2))
LANGUAGE plpgsql
AS $$
BEGIN
    -- Get the promotion name and calculate total sales
    SELECT p.promo_name, SUM(s.sales) INTO promo_name, total_sales
    FROM sales AS s
    JOIN promotions AS p
	ON s.promo_id = p.promo_id
    WHERE s.promo_id = p.promo_id
    GROUP BY p.promo_name;

    -- If no sales are found, set total_sales to 0 and promo_name to 'Unknown'
    IF total_sales IS NULL THEN
        total_sales := 0;
        promo_name := 'Unknown';
    END IF;

    -- Output the result
    RAISE NOTICE 'Total sales for promotion "%" (ID %): %', promo_name, promo_id, total_sales;
END;
$$;

- Calling the procedure (example with `promo_id` 2):

In [ ]:
-- Call the enhanced procedure for promotion ID 2
CALL calculate_total_sales_for_promotion_with_name(2, NULL, NULL);

## Trigger

### 1. Automatically Assign `promo_id` on Insert
This trigger automatically assigns the appropriate `promo_id` when inserting rows into the `sales` table based on the product category and order date.

In [ ]:
CREATE OR REPLACE FUNCTION assign_promo_id()
RETURNS TRIGGER
LANGUAGE plpgsql
AS $$
BEGIN
    -- Find the applicable promotion for the product category and order date
    SELECT promo_id INTO NEW.promo_id
    FROM promotions p
    JOIN products pr ON p.applicable_category = pr.category
    WHERE pr.product_id = NEW.product_id
      AND NEW.order_date BETWEEN p.start_date AND p.end_date
    LIMIT 1;

    -- If no promotion is found, set promo_id to 1 (No Promotion)
    IF NEW.promo_id IS NULL THEN
        NEW.promo_id := 1;
    END IF;

    RETURN NEW;
END;
$$;

-- Trigger Definition
CREATE TRIGGER trg_assign_promo_id
BEFORE INSERT ON sales
FOR EACH ROW
EXECUTE FUNCTION assign_promo_id();

- Example of Trigger in Action:

In [ ]:
INSERT INTO sales (row_id, order_id, order_date, ship_date, customer_id, postal_code, product_id, sales)
VALUES (1001, 'CA-2023-123456', '2023-08-15', '2023-08-20', 'CUST123', '12345', 'PROD456', 100.00);

-- The trigger will automatically assign the respective promo_id if the order_date falls within the promotion period.

# View

### 1. sales_overview
Creating a view in PostgreSQL is a great way to simplify complex queries and provide a reusable, virtual table for analysis. Below is an example of a view that combines data from the sales, customers, products, locations, promotions, and ship_modes tables to provide a comprehensive overview of sales data.

In [ ]:
CREATE OR REPLACE VIEW sales_overview AS
SELECT
    s.row_id,
    s.order_id,
    s.order_date,
    s.ship_date,
    s.sales,
    c.customer_id,
    c.customer_name,
    c.segment AS customer_segment,
    p.product_id,
    p.product_name,
    p.category AS product_category,
    p.sub_category AS product_sub_category,
    l.postal_code,
    l.city,
    l.state,
    l.region,
    l.country,
    sm.ship_mode_name,
    pr.promo_name,
    pr.discount_pct,
    pr.start_date AS promo_start_date,
    pr.end_date AS promo_end_date
FROM
    sales s
    JOIN customers c ON s.customer_id = c.customer_id
    JOIN products p ON s.product_id = p.product_id
    JOIN locations l ON s.postal_code = l.postal_code
    JOIN ship_modes sm ON s.ship_mode_id = sm.ship_mode_id
    LEFT JOIN promotions pr ON s.promo_id = pr.promo_id;

### Explanation of the View

#### Tables Joined:
- **`sales`**: Core table containing transactional data.
- **`customers`**: Provides customer details (`customer_name`, `segment`).
- **`products`**: Provides product details (`product_name`, `category`, `sub_category`).
- **`locations`**: Provides geographical details (`city`, `state`, `region`, `country`).
- **`ship_modes`**: Provides shipping method details (`ship_mode_name`).
- **`promotions`**: Provides promotion details (`promo_name`, `discount_pct`, `start_date`, `end_date`).

#### Columns Included:
- **Sales data**: `row_id`, `order_id`, `order_date`, `ship_date`, `sales`.
- **Customer data**: `customer_id`, `customer_name`, `customer_segment`.
- **Product data**: `product_id`, `product_name`, `product_category`, `product_sub_category`.
- **Location data**: `postal_code`, `city`, `state`, `region`, `country`.
- **Shipping data**: `ship_mode_name`.
- **Promotion data**: `promo_name`, `discount_pct`, `promo_start_date`, `promo_end_date`.

#### Left Join for Promotions:
A `LEFT JOIN` is used for the `promotions` table to ensure that sales without promotions are still included in the view.

In [ ]:
-- Example Queries Using the View:

-- Get all sales for the "Office Supplies" category
SELECT *
FROM sales_overview
WHERE product_category = 'Office Supplies';

-- Get total sales by region
SELECT region, SUM(sales) AS total_sales
FROM sales_overview
GROUP BY region;


-- Get sales with promotions applied
SELECT *
FROM sales_overview
WHERE promo_name IS NOT NULL;


## Final Notes

### Key Achievements
1. **Stored Procedures**:
   - Procedures like `calculate_total_sales_for_promotion` and `update_promo_id_for_category` encapsulate business logic for calculating sales and updating promotions.
   - These procedures make it easy to perform repetitive tasks and ensure consistency across the database.

2. **Triggers**:
   - The `assign_promo_id` trigger automates the assignment of promotions during data insertion, ensuring that the correct `promo_id` is applied based on the product category and order date.

3. **Views**:
   - The `sales_overview` view provides a comprehensive and reusable summary of sales data, combining information from multiple tables into a single virtual table.
   - This simplifies complex queries and makes it easier to generate reports and insights.

### Future Enhancements
- **Indexes**: Add indexes on frequently queried columns (e.g., `order_date`, `customer_id`) to improve query performance.
- **Data Validation**: Add `CHECK` constraints to enforce data quality (e.g., `sales > 0`).
- **Audit Columns**: Include `created_at` and `updated_at` columns to track changes over time.
- **Advanced Analytics**: Create additional views and procedures to support advanced analytics, such as sales trends and customer segmentation.